In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Concatenate, Input, Lambda, Reshape
from keras.layers import Embedding, LSTM, TimeDistributed
from keras import Model
from keras.utils.vis_utils import plot_model
from keras import backend as K

def rescale(x):
	input_value = x[0]
	embedding = x[1]
	## force cat_code 0 represent missing data
	mask = K.cast(input_value == -999, dtype=K.floatx())
	return embedding * mask + tf.multiply(embedding, input_value) * (1 - mask)

In [8]:
categorical_vars = {"z": 2}
continuous_dim = 1
needs_embed = [False]

embedding_size = 5

ins = []
sub_models = []

for cat_var, k in categorical_vars.items():
	print("cat")
	_input = Input(shape=(1,), name=cat_var)
	ins.append(_input)
	_cat_embed = Embedding(k+1, embedding_size, input_length=1)(_input)
	_cat_embed = Reshape(target_shape=(embedding_size,))(_cat_embed)
	# use case
	for i in range(3):
		_cat_embed = Dense(10)(_cat_embed)
	z = Dense(k-1)(_cat_embed)
	sub_models.append(z)

scaled_embeds = []
for i in range(continuous_dim):
	print("cont")
	_input = Input(shape=(1,))
	ins.append(_input)
	if needs_embed[i]:
		_input_indct = Input(shape=(1,))
		ins.append(_input_indct)
		_indct_embed = Embedding(2, embedding_size, input_length=1)(_input_indct)
		_indct_embed = Reshape(target_shape=(embedding_size,))(_indct_embed)
		_input = Lambda(rescale)((_input, _indct_embed))

	sub_models.append(_input)

x = Concatenate()(sub_models)
# for i in range(4):
# 	x = Dense(50)(x)
out = Dense(1, name="weights")(x)
model = Model(inputs=ins, outputs=out)

cat
cont


In [14]:
df = pd.read_csv("./simlated_data.csv")
df["z"] = df["z"].fillna(0)
df["z"] = df["z"]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       5000 non-null   float64
 1   z       5000 non-null   float64
 2   y       5000 non-null   float64
dtypes: float64(3)
memory usage: 117.3 KB


In [15]:
((df["y"] - df["y"].mean()) ** 2).mean()

30.805468629556415

In [19]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mean_squared_error")

In [22]:
model.fit(
	x=[df["z"], df["x"]], y=df["y"], epochs=10, validation_split=0.1, verbose=0
)

In [23]:
model.get_layer(name="weights").get_weights()
# z, x, bias

[array([[0.17955436],
        [0.9430799 ]], dtype=float32),
 array([0.09139689], dtype=float32)]

In [29]:
yh = model.predict(x=[df["z"], df["x"]]).reshape(-1)

157/157 [==============================] - 0s 1ms/step


In [32]:
((yh - df["y"].values)**2).mean()

6.277243238398964